In [1]:
from datetime import datetime
import boto3
import sagemaker

client = boto3.client(service_name='sagemaker')
now = datetime.now()
current_time = now.strftime("%Y-%m-%d-%H-%M-%S")

region = boto3.Session().region_name
account_id = boto3.client('sts').get_caller_identity()['Account']
project_name = 'sac-emails'
container = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{project_name}:latest'


#### Create Estimator

In [ ]:
model = sagemaker.estimator.Estimator(
    image_uri=container,
    role='arn:aws:iam::296025910508:role/datascience-sagemaker-s3-redshift',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    output_path=f's3://sagemaker-us-east-1-296025910508/{project_name}/training_jobs/'
)

#### Fit

In [ ]:
train_input = 's3://sagemaker-us-east-1-296025910508/sac-emails/input/training_data/poc_sac.parquet'
# train_input = 'file://../data/poc_sac.parquet'

model.fit(
    inputs={"training": train_input},
    job_name=f'{project_name}-{current_time}'
)

#### Deploy

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=f'{project_name}-endpoint',
    model_name=f'{project_name}-{current_time}-model'
)

#### Batch Transform

In [ ]:
create_batch = client.create_transform_job(
    TransformJobName='sac-emails-{}'.format(current_time),
    ModelName='sac-emails-2022-02-08-15-44-01-model',
    # MaxConcurrentTransforms=0,
    # MaxPayloadInMB=20,
    # BatchStrategy='MultiRecord',
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://sagemaker-us-east-1-296025910508/sac-emails/input/payload.csv'
            }
        },
        'ContentType': 'text/csv'
        # 'SplitType': 'Line'
    },
    TransformOutput={
        'S3OutputPath': 's3://sagemaker-us-east-1-296025910508/sac-emails/output/'
    },
    TransformResources={
        'InstanceType': 'ml.m5.xlarge',
        'InstanceCount': 1
    }
)

In [2]:
# attached_estimator = sagemaker.estimator.Estimator.attach(training_job_name = 'sac-emails-2022-05-16-20-14-43')

attached_estimator = sagemaker.estimator.Estimator.attach(training_job_name = 'sac-emails-2022-16-05-20-59-16')



2022-05-17 00:25:19 Starting - Preparing the instances for training
2022-05-17 00:25:19 Downloading - Downloading input data
2022-05-17 00:25:19 Training - Training image download completed. Training in progress.
2022-05-17 00:25:19 Uploading - Uploading generated training model
2022-05-17 00:25:19 Completed - Training job completed


In [4]:
predictor = attached_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.large',
    endpoint_name=f'{project_name}-endpoint',
    model_name=f'{project_name}-{current_time}-model'
)

Using already existing model: sac-emails-2022-05-17-09-54-11-model


--------------------------------------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint sac-emails-endpoint: Failed. Reason:  The customer:primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
import json, boto3
sm_client=boto3.client('sagemaker-runtime')

payload = '''
[
    { 
    "message":"Me arrependi da compra e quero cancelar",
    "problem":"Trocar, cancelar ou devolver",
    "exemplo":"exemplo"
    }

]
'''

response = sm_client.invoke_endpoint(
    EndpointName='sac-emails-endpoint-2',
    Body=payload,
    ContentType='application/json')

response.get('Body').read().decode('utf-8')